In [0]:
import requests
import pandas as pd
import json
import time
import re
import logging
import os
from datetime import datetime, timedelta

In [0]:
import requests
from pyspark.sql import DataFrame
import pyspark.sql.functions as F
from pyspark.sql.types import *
import json
import time
import re
import logging
from pyspark.sql.window import Window
import os
import re

In [0]:
dbutils.widgets.dropdown(name="include_historical_data", defaultValue="False", choices=["True", "False"])
dbutils.widgets.text(name = "history_years_param", defaultValue="3")

history_years = eval(dbutils.widgets.get("history_years_param"))
include_historical_data = eval(dbutils.widgets.get("include_historical_data"))
print("history_years: ", history_years)
print("include_historical_data: ", include_historical_data)

In [0]:
logger = logging.getLogger(name= os.path.basename(dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()))
logger.propagate = False
logger.setLevel(logging.INFO)

stream_handler = logging.StreamHandler()

formatter = logging.Formatter(fmt= "{asctime} - {name} - {levelname} --> {message}", style= '{', datefmt='%Y-%m-%d %H:%M:%S')
stream_handler.setFormatter(formatter)

logger.handlers.clear()    
logger.addHandler(stream_handler)

logger.info("Notebook run started")

In [0]:
header = {'User-Agent' : "rkumar11@mscience.com"}

In [0]:
tickers_str_with_region = """ 
RRR-US
UPWK-US
DPZ-US
SBUX-US
BLMN-US
CHWY-US
DRVN-US
TEAM-US
AMZN-US
WIX-US
ORLY-US
BABA-US
PETS-US
ALK-US
OGS-US
RGS-US
EA-US
KMX-US
PDD-US
RAVE-US
IDT-US
GT-US
NKE-US
ULCC-US
KR-US
FDX-US
BJRI-US
DTE-US
ACI-US
LOCO-US
TSCO-US
ADBE-US
BYD-US
ODP-US
SFIX-US
SSTK-US
CAKE-US
DECK-US
PGR-US
EXR-US
T-US
WW-US
COUR-US
UPS-US
MNSO-US
GDDY-US
FVRR-US
EWCZ-US
BOOT-US
SKT-US
LOVE-US
PBPB-US
CUBE-US
VG-US
NTES-US
MNDY-US
AAPL-US
CCL-US
PLAY-US
UNFI-US
TALK-US
COIN-US
BBBY-US
CHTR-US
CVX-US
XOM-US
GWW-US
PETS-US
DENN-US
SPG-US
OGS-US
MED-US
TDUP-US
UPWK-US
BRCC-US
HIMS-US
SGHC-US
FUBO-US
IAC-US
BRK.A-US
NSA-US
GT-US
"""

In [0]:
# Output list of tickers 1.when revenue is not available 2.In cases where any exception occurs(delisted etc)
rev_bif_not_found = []

# processed tickers
processed_tickers = []

# Final result df with the required schema
final_result_df = pd.DataFrame(
    columns=[
        "Ticker",
        "Period",
        "Period_Startdate",
        "Period_Enddate",
        "Period_Reportdate",
        "_",
        "Segment_Identifier",
        "KPI",
        "Value",
        "Revison_Date",
        "Original_Value",
        "Source",
    ]
)

In [0]:
class InvokeCorrespondingDeco:
    _map_funcs = {
                "get_input_tickers" : "get_input_tickers_pyspark",
                "get_cik_lookup_df" : "get_cik_lookup_df_pyspark",
                "get_company_facts" : "get_company_facts_pyspark",
                "create_stmnts_combined_df" : "create_stmnts_combined_df_pyspark",
                "remove_inconsistencies" : "remove_inconsistencies_pyspark",
                "save_latest_q1_q3" : "save_latest_q1_q3_pyspark",
                "assign_relation_between_q_and_fy" : "assign_relation_between_q_and_fy_pyspark",
                "get_q4_rows" : "get_q4_rows_pyspark",
                "get_fy_all_q_union_df" : "get_fy_all_q_union_df_pyspark",
                "period_corrected" : "period_corrected_pyspark",
                "get_base_sheet_format" : "get_base_sheet_format_pyspark",
                "get_not_in_base_sheet_rows"   : "get_not_in_base_sheet_rows_pyspark",
            }
    
    __DECORATOR_LOGS = {k:{"only_in_pandas_df": None, "only_in_pyspark_df": None} for k, v in _map_funcs.items()}

    @staticmethod
    def invoke_corresponding(func):

        def wrap(*args, **kwargs):
            # map functions functions to pyspark functions
            
            print(f'Comparing the pandas function "{func.__name__}" against the pyspark function "{InvokeCorrespondingDeco._map_funcs[func.__name__]}" ')

            pandas_output = func(*args, **kwargs)
            pandas_output_copy = pandas_output.copy()
            
            #create args and keyword args for the corresponding pyspark function
            pyspark_args = [spark.createDataFrame(arg) if isinstance(arg, pd.DataFrame) else arg for arg in args]
            pyspark_kwargs = {k: spark.createDataFrame(v) if isinstance(v, pd.DataFrame) else v for k, v in kwargs.items()}

            #calling the corresponding pyspark function
            pyspark_output = globals()[InvokeCorrespondingDeco._map_funcs[func.__name__]](*pyspark_args, **pyspark_kwargs)
            
            if isinstance(pandas_output, pd.DataFrame):
                cols_only_in_pandas = set(pandas_output.columns) - set(pyspark_output.columns)
                cols_only_in_pyspark = set(pyspark_output.columns) - set(pandas_output.columns)
                if cols_only_in_pandas or cols_only_in_pyspark:
                    print(f"Schema difference between found between pandas and pyspark output")
                    print(f"Columns only in pandas output: {cols_only_in_pandas}")
                    print(f"Columns only in pyspark output: {cols_only_in_pyspark}")
                else:
                    print("Schema is same between pandas and pyspark output")
                merge_keys =  [col for col in pyspark_output.columns if re.search(col.lower(), "start|end")]
                pyspark_output_pd = pyspark_output.toPandas()

                if pandas_output_copy.empty:
                    print("pandas df is empty")
                elif pyspark_output.isEmpty():
                    print("pyspark df is empty")
                elif pandas_output_copy.empty and pyspark_output.isEmpty():
                    print("Both pandas and pyspark df are empty")
                else:
                    joined_df = pd.merge(
                        left = pandas_output_copy, 
                        right = pyspark_output_pd,
                        on= merge_keys,
                        suffixes=("_pandas", "_pyspark"),
                        how= "outer",
                        indicator=True
                    )
                    InvokeCorrespondingDeco.__DECORATOR_LOGS[func.__name__]["only_in_pandas_df"] = joined_df[joined_df["_merge"] == "left_only"]
                    InvokeCorrespondingDeco.__DECORATOR_LOGS[func.__name__]["only_in_pyspark_df"] = joined_df[joined_df["_merge"] == "right_only"]

            elif isinstance(pandas_output, list) or isinstance(pandas_output, dict):
                if not pandas_output == pyspark_output:
                    print("Pandas and pyspark outputs are different")
                    # print(pandas_output)
                    # print(pyspark_output)
                else:
                    print("Pandas and pyspark outputs are same")
            print("-"*150)
            return pandas_output
        return wrap
    
    @staticmethod
    def get_deco_logs():
        return __DECORATOR_LOGS

    


In [0]:
def debug(func):
    def debug_wrap(*args, **kwargs):
        print(f"Calling {func.__name__} with args {args} and kwargs {kwargs}")
        return func(*args, **kwargs)
    return debug_wrap

In [0]:
# @debug
def get_input_tickers_pyspark(tickers_str_with_region:str) -> list[str]:
    # segregate tickers by region
    tickers_list_us_region = [
        ticker.lstrip().rstrip()
        for ticker in tickers_str_with_region.split("\n")
        if ticker != "" and ticker.endswith("-US")
    ]
    # print(tickers_list_us_region)
    tickers_list_int_region = [
        ticker.lstrip().rstrip()
        for ticker in tickers_str_with_region.split("\n")
        if ticker != "" and not ticker.endswith("-US")
    ]
    # print(tickers_list_int_region)
    # get the ticker part only
    tickers_list = [ticker.split("-")[0] for ticker in tickers_list_us_region]

    # handle cases with duplicate entries
    tickers_to_check = [ticker for ticker in set(tickers_list)]

    # add international tickers to flag
    rev_bif_not_found.extend([t for t in list(set(tickers_list_int_region)) if t != ""])
    # print(rev_bif_not_found)
    return tickers_to_check

# # @debug
def get_cik_lookup_df_pyspark() -> DataFrame:
    # Get CIK lookup table through company tickers endpoint
    cik_map = requests.get(
        "https://www.sec.gov/files/company_tickers.json", headers=header
    )
    # print(cik_map.status_code)
    cik_map_df = spark.createDataFrame(
        cik_map.json().values(),
        schema=StructType(
            [
                StructField("cik_str", StringType(), True),
                StructField("ticker", StringType(), True),
                StructField("title", StringType(), True),
            ]
        ),
    )
    # padding to get 10 digit cik
    cik_map_df = cik_map_df.withColumn("cik_str", F.lpad(F.col("cik_str"), 10, "0"))
    return cik_map_df

# @debug
def get_company_facts_pyspark(cik:str) -> dict:
    # company facts endpoint
    URL = f"https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json"
    comp_facts = requests.get(URL, headers=header)
    time.sleep(1)
    comp_facts_dict = json.loads(comp_facts.text)
    return comp_facts_dict

# @debug
def create_stmnts_combined_df_pyspark(comp_facts_dict: dict, history_years: int= 3):
    # stmnts_to_scan= [stmnt for stmnt in comp_facts_dict['facts']['us-gaap'].keys() if re.search("revenue", stmnt.lower())]
    if "us-gaap" not in comp_facts_dict["facts"].keys():
        return None
    else:
        stmnts_to_scan = [
            "RevenueFromContractWithCustomerExcludingAssessedTax",
            "RevenueFromContractWithCustomerIncludingAssessedTax",
            # 'RevenueFromContractsWithCustomers',
            "Revenues",
        ]
        stmnts_combined_df_op = spark.createDataFrame(
            [],
            schema=StructType(
                [
                    StructField("end", StringType(), True),
                    StructField("fp", StringType(), True),
                    StructField("start", StringType(), True),
                    StructField("filed", StringType(), True),
                    StructField("time_period_months", FloatType(), True),
                    StructField("val", LongType(), True),
                    StructField("orig_stmnt", StringType(), True),
                ]
            ),
        )
        for stmnt in stmnts_to_scan:
            try:
                stmnt_df = spark.createDataFrame(
                    comp_facts_dict["facts"]["us-gaap"][stmnt]["units"]["USD"]
                )
                # check must have columns -- if any statements lacks these skip it
                if not (
                    {"start", "end", "fp", "val", "filed"}.issubset(
                        set(stmnt_df.columns)
                    )
                ):
                    continue
                # basic transformations and filtering
                else:
                    stmnt_df = (
                        stmnt_df.withColumn("start", F.col("start").cast(DateType()))
                        .withColumn("end", F.col("end").cast(DateType()))
                        .withColumn("filed", F.col("filed").cast(DateType()))
                        .withColumn("val", F.col("val").cast(LongType()))
                        .withColumn(
                            "time_period_months",
                            F.round(F.months_between(F.col("end"), F.col("start")), 2),
                        )
                        .withColumn("orig_stmnt", F.lit(stmnt))
                    )
                    stmnt_df = (
                        stmnt_df
                        # filtering out records that are not within the last history_years --IMPORTANT
                        .filter(
                            f" start > date_sub(current_date(), {history_years*366}) "
                        )
                        # filter out rows that are not quarterly/yearly
                        .filter(
                            "  (time_period_months between 2.7 and 4) or (time_period_months between 11 and 13) "
                        )
                    )
                    stmnts_combined_df_op = stmnts_combined_df_op.unionByName(
                        stmnt_df.select(
                            "start",
                            "end",
                            "fp",
                            "time_period_months",
                            "val",
                            "filed",
                            "orig_stmnt",
                        )
                    )
            except KeyError as ke:
                # logger.error(f"KeyError: {ke}")
                continue
    return stmnts_combined_df_op.select(
        "start", "end", "fp", "time_period_months", "val", "filed", "orig_stmnt"
    )

# @debug
def remove_inconsistencies_pyspark(stmnts_combined_df: DataFrame) -> DataFrame:
    stmnts_combined_df_cleansed = (
        stmnts_combined_df.withColumn(
            "max_rev", F.max("val").over(Window.partitionBy("start", "end", "fp"))
        )
        .withColumn(
            "rank_rev",
            F.row_number().over(
                Window.partitionBy("start", "end", "fp", "max_rev").orderBy(
                    F.desc("val")
                )
            ),
        )
        .filter(F.col("rank_rev") == 1)
        .drop("max_rev", "rank_rev")
        # Filtering out ambigous cases
        .filter(
            r""" fp is not null and not(time_period_months < 11 and fp == "FY") and not(time_period_months > 11 and fp rlike "^Q\\d{1}$") """
        )
        .groupBy("start", "end", "time_period_months", "fp")
        .agg(
            F.count(F.lit(1)).alias("count"),
            F.max("val").alias("val"),
            F.collect_list(F.col("filed")).alias("list_filed"),
        )
        # filtering out rows with less count when start and end are same but fp is different
        .withColumn(
            "rn",
            F.rank().over(Window.partitionBy("start", "end").orderBy(F.desc("count"))),
        )
        .filter(F.col("rn") == 1)
        # filtering out rows with less count when fp is same and the respective start and end dates are very close in terms of days(inconsistent data)
        .withColumn("start_month", F.trunc(F.col("start"), "month"))
        .withColumn("end_month", F.trunc(F.col("end"), "month"))
        .withColumn(
            "rnk_m",
            F.rank().over(
                Window.partitionBy("start_month", "end_month", "fp").orderBy(
                    F.col("count").desc()
                )
            ),
        )
        .filter(F.col("rnk_m") == 1)
        .withColumn("week_end", F.date_trunc("week", F.col("end")))
        .withColumn("week_start", F.date_trunc("week", F.col("start")))
        .withColumn(
            "rnk_w",
            F.rank().over(
                Window.partitionBy("week_end", "week_start", "fp").orderBy(
                    F.col("count").desc()
                )
            ),
        )
        .filter(F.col("rnk_w") == 1)
        .withColumn("filed", F.explode("list_filed"))
        .drop(
            "rn",
            "rnk_m",
            "rnk_w",
            "week_end",
            "week_start",
            "start_month",
            "end_month",
            "count",
            "list_filed",
        )
    )
    return stmnts_combined_df_cleansed

# @debug
def save_latest_q1_q3_pyspark(stmnts_combined_df_cleansed: DataFrame) -> DataFrame:
    # save the latest q1-q3 for union later
    max_end_fy = (
        stmnts_combined_df_cleansed.filter(" fp == 'FY' ")
        .agg(F.max("end"))
        .first()[0]
    )
    latest_q1_q3 = stmnts_combined_df_cleansed.filter(F.col("end") > max_end_fy)
    return latest_q1_q3

# @debug
def assign_relation_between_q_and_fy_pyspark(stmnts_combined_df_cleansed):
    fy_rows = stmnts_combined_df_cleansed.filter(" fp == 'FY' ").selectExpr(
        "start as fy_start", "end as fy_end"
    )

    stmnts_combined_df_with_rel = (
        stmnts_combined_df_cleansed.crossJoin(fy_rows)
        .filter(" start >= fy_start and end <= fy_end")
        .withColumn(
            "q1_q3_fy_rel", F.concat_ws(" ", F.col("fy_start"), F.col("fy_end"))
        )
    )

    return stmnts_combined_df_with_rel

# @debug
def get_q4_rows_pyspark(stmnts_combined_df_with_rel: DataFrame) -> DataFrame:
    q4_rows_df = (
        stmnts_combined_df_with_rel.withColumn(
            "q1_q3_fy_rev", F.sum("val").over(Window.partitionBy("q1_q3_fy_rel"))
        )
        .withColumn(
            "q1_q3_rev",
            F.when(F.col("fp") == "FY", F.col("q1_q3_fy_rev") - F.col("val")).otherwise(
                None
            ),
        )
        .withColumn(
            "q4_rev",
            F.when(F.col("fp") == "FY", F.col("val") - F.col("q1_q3_rev")).otherwise(
                None
            ),
        )
        .withColumn("q4_rev", F.max("q4_rev").over(Window.partitionBy("q1_q3_fy_rel")))
        .withColumn(
            "rnk_q1_q3_fy_rel",
            F.rank().over(
                Window.partitionBy("q1_q3_fy_rel").orderBy(F.col("end").desc())
            ),
        )
        .withColumn(
            "q4_start",
            F.when(
                F.col("rnk_q1_q3_fy_rel") == 2, F.date_add(F.col("end"), 1)
            ).otherwise(None),
        )
        .withColumn(
            "q4_end",
            F.when(F.col("rnk_q1_q3_fy_rel") == 1, F.col("end")).otherwise(None),
        )
        .withColumn(
            "q4_filed",
            F.when(F.col("rnk_q1_q3_fy_rel") == 1, F.col("filed")).otherwise(None),
        )
        .withColumn(
            "running_max_q4_start",
            F.max(F.col("q4_start")).over(
                Window.rowsBetween(Window.unboundedPreceding, Window.currentRow)
            ),
        )
        .withColumn(
            "running_max_q4_end",
            F.max(F.col("q4_end")).over(
                Window.rowsBetween(Window.unboundedPreceding, Window.currentRow)
            ),
        )
        .withColumn(
            "running_max_q4_filed",
            F.max(F.col("q4_filed")).over(
                Window.rowsBetween(Window.unboundedPreceding, Window.currentRow)
            ),
        )
        .withColumn(
            "time_period_months_q4",
            F.round(
                F.months_between(
                    F.col("running_max_q4_end"), F.col("running_max_q4_start")
                ),
                2,
            ),
        )
        .withColumn(
            "time_period_months_q4",
            F.round(
                F.months_between(
                    F.col("running_max_q4_end"), F.col("running_max_q4_start")
                ),
                2,
            ),
        )
        .filter(" time_period_months_q4 between 2.5 and 4 ")
        .withColumn("fp", F.lit("Q4"))
        .selectExpr(
            "running_max_q4_start as start",
            "running_max_q4_end as end",
            "fp",
            "q4_rev as val",
            "running_max_q4_filed as filed",
        )
        .distinct()
    )
    return q4_rows_df

# @debug
def get_fy_all_q_union_df_pyspark(stmnts_combined_df_with_rel: DataFrame, latest_q1_q3: DataFrame, q4_rows_df: DataFrame) -> DataFrame:
    return (
        stmnts_combined_df_with_rel.select("start", "end", "fp", "val", "filed")
        .unionByName(q4_rows_df.select("start", "end", "fp", "val", "filed"))
        .unionByName(latest_q1_q3.select("start", "end", "fp", "val", "filed"))
    )

# @debug
def period_corrected_pyspark(fy_all_q_union_df: DataFrame) -> DataFrame:
    fy_all_q_union_df_fy_rows = fy_all_q_union_df.filter(
        F.col("fp") == "FY"
    ).selectExpr(["start as fy_start", "end as fy_end"])
    fy_all_q_union_latest_q_rows = save_latest_q1_q3(fy_all_q_union_df)
    upto_latest_fy_pc_df = (
        fy_all_q_union_df.crossJoin(fy_all_q_union_df_fy_rows)
        .filter(" start >= fy_start and end <= fy_end")
        .withColumn(
            "Period",
            F.when(
                F.col("fp").isin(["FY"]),
                F.concat_ws(
                    "",
                    F.col("fp"),
                    F.regexp_extract(
                        F.year(F.date_trunc("year", F.col("fy_end"))).cast(
                            StringType()
                        ),
                        "^\\d{2}(\\d{2})$",
                        1,
                    ),
                ),
            ).otherwise(
                F.concat_ws(
                    "",
                    F.reverse(F.col("fp")),
                    F.regexp_extract(
                        F.year(F.date_trunc("year", F.col("fy_end"))).cast(
                            StringType()
                        ),
                        "^\\d{2}(\\d{2})$",
                        1,
                    ),
                )
            ),
        )
        .select("start", "end", "fp", "Period", "val", "filed")
    )
    max_end_fy = fy_all_q_union_df_fy_rows.selectExpr(" max(fy_end) as max_end_fy ")
    try:
        period_val_max_end_fy = (
            upto_latest_fy_pc_df.join(
                max_end_fy, on=F.col("end") == F.col("max_end_fy"), how="inner"
            )
            .selectExpr("regexp_extract(Period, r'^.*(\\d{2,}).*$', 1)")
            .first()[0]
        )
        period_for_latest_q_rows = str(int(period_val_max_end_fy) + 1)
        fy_all_q_union_latest_q_pc_rows = fy_all_q_union_latest_q_rows.withColumn(
            "Period",
            F.concat_ws("", F.reverse(F.col("fp")), F.lit(period_for_latest_q_rows)),
        ).select("start", "end", "fp", "Period", "val", "filed")
    except TypeError:
        return upto_latest_fy_pc_df
    return upto_latest_fy_pc_df.unionByName(fy_all_q_union_latest_q_pc_rows)

# @debug
def get_base_sheet_format_pyspark(fy_all_q_union_df: DataFrame, Ticker: str) -> DataFrame:
    # base sheet format
    fy_all_q_union_bsf_df = (
        fy_all_q_union_df.withColumn("Ticker", F.lit(f"{Ticker}" + " " + "US Equity"))
        .withColumn("Period_Startdate", F.date_format(F.col("start"), "M/d/y"))
        .withColumn("Period_Reportdate", F.date_format(F.col("filed"), "M/d/y"))
        .withColumn("Period_Enddate", F.date_format(F.col("end"), "M/d/y"))
        .withColumn("Value", F.format_number(F.col("val"), "###,###"))
        .withColumn("KPI", F.lit("rev_Topline"))
        .withColumn("_", F.lit(""))
        .withColumn("Segment_Identifier", F.lit(""))
        .withColumn("Revison_Date", F.lit(""))
        .withColumn("Original_Value", F.lit(""))
        .withColumn(
            "Source",
            F.when(
                F.col("fp").isin(["FY", "Q4"]),
                F.concat_ws(" ", F.lit("10-K"), F.col("Period")),
            ).otherwise(F.concat_ws(" ", F.lit("10-Q"), F.col("Period"))),
        )
    )

    fy_all_q_union_bsf_df = fy_all_q_union_bsf_df.select(
        [
            "Ticker",
            "Period",
            "Period_Startdate",
            "Period_Enddate",
            "Period_Reportdate",
            "_",
            "Segment_Identifier",
            "KPI",
            "Value",
            "Revison_Date",
            "Original_Value",
            "Source",
        ]
    ).filter((F.col("Period").rlike("Q|F")) & (F.col("val") > 0))
    return fy_all_q_union_bsf_df

# @debug
def get_not_in_base_sheet_rows_pyspark(fy_all_q_union_df: DataFrame, Ticker: str) -> DataFrame:
    not_in_base_sheet_df = (
        F.broadcast(
            fy_all_q_union_df.withColumn(
                "period_sub_str", F.col("period").substr(0, 2)
            ).alias("a")
        )
        .join(
            table("bronze.internal_products.financials_cians_raw")
            .filter(F.col("ticker") == Ticker + " US Equity")
            .filter(F.col("kpi") == "rev_Topline")
            .filter(F.col("value").isNotNull())
            .select("period", "period_startdate", "period_enddate")
            .withColumn("period_sub_str", F.col("period").substr(0, 2))
            .alias("b"),
            on=(F.col("a.Period_Startdate") == F.col("b.period_startdate"))
            & (F.col("a.Period_Enddate") == F.col("b.period_enddate"))
            & (F.col("a.period_sub_str") == F.col("b.period_sub_str")),
            how="left_anti",
        )
        .drop("period_sub_str")
    )
    return not_in_base_sheet_df

In [0]:
# @InvokeCorrespondingDeco.invoke_corresponding
def get_input_tickers(tickers_str_with_region:str) -> list[str]:
    # segregate tickers by region
    tickers_list_us_region = [
        ticker.lstrip().rstrip()
        for ticker in tickers_str_with_region.split("\n")
        if ticker != "" and ticker.endswith("-US")
    ]
    # print(tickers_list_us_region)
    tickers_list_int_region = [
        ticker.lstrip().rstrip()
        for ticker in tickers_str_with_region.split("\n")
        if ticker != "" and not ticker.endswith("-US")
    ]
    # print(tickers_list_int_region)
    # get the ticker part only
    tickers_list = [ticker.split("-")[0] for ticker in tickers_list_us_region]

    # handle cases with duplicate entries
    tickers_to_check = [ticker for ticker in set(tickers_list)]

    # add international tickers to flag
    rev_bif_not_found.extend([t for t in list(set(tickers_list_int_region)) if t != ""])
    # print(rev_bif_not_found)
    return tickers_to_check

# @InvokeCorrespondingDeco.invoke_corresponding
def get_cik_lookup_df() -> pd.DataFrame:
    # Get CIK lookup table through company tickers endpoint
    cik_map = requests.get(
        "https://www.sec.gov/files/company_tickers.json", headers=header
    )
    cik_map_json = cik_map.json()
    cik_map_list = [
        {
            "ticker": v["ticker"],
            #padding to get 10 digit cik 
            "cik": str(v["cik_str"]).zfill(10),
            "title": v["title"]
        }
        for v in cik_map_json.values()
    ]
    cik_map_df = pd.DataFrame(cik_map_list)
    return cik_map_df

# @InvokeCorrespondingDeco.invoke_corresponding
def get_company_facts(cik:str) -> dict:
    # company facts endpoint
    URL = f"https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json"
    comp_facts = requests.get(URL, headers=header)
    # time.sleep(1)
    comp_facts_dict = json.loads(comp_facts.text)
    return comp_facts_dict

# @InvokeCorrespondingDeco.invoke_corresponding
def create_stmnts_combined_df(comp_facts_dict: dict, history_years: int= 3) -> pd.DataFrame | None:
    # stmnts_to_scan= [stmnt for stmnt in comp_facts_dict['facts']['us-gaap'].keys() if re.search("revenue", stmnt.lower())]
    if "us-gaap" not in comp_facts_dict["facts"].keys():
        return None
    else:
        stmnts_to_scan = [
            "RevenueFromContractWithCustomerExcludingAssessedTax",
            "RevenueFromContractWithCustomerIncludingAssessedTax",
            # 'RevenueFromContractsWithCustomers',
            "Revenues",
        ]
        stmnts_combined_df_op = pd.DataFrame([], columns= ["start", "end", "fp", "time_period_months", "val", "filed", "orig_stmnt"])
        for stmnt in stmnts_to_scan:
            try:
                stmnt_df = pd.DataFrame(
                    comp_facts_dict["facts"]["us-gaap"][stmnt]["units"]["USD"]
                )
                # check must have columns -- if any statements lacks these skip it
                if not (
                    {"start", "end", "fp", "val", "filed"}.issubset(
                        set(stmnt_df.columns)
                    )
                ):
                    continue
                # basic transformations and filtering
                else:
                    stmnt_df = stmnt_df.drop_duplicates(subset=["start", "end", "fp", "val"])

                    stmnt_df['start'] = pd.to_datetime(stmnt_df['start']).dt.date
                    stmnt_df['end'] = pd.to_datetime(stmnt_df['end']).dt.date
                    stmnt_df['filed'] = pd.to_datetime(stmnt_df['filed']).dt.date
                    stmnt_df['val'] = stmnt_df['val'].astype(int)
                    #for debugging
                    stmnt_df['orig_stmnt'] = stmnt

                    def get_time_period_months(row):
                        row['time_period_months'] = (row['end'].year - row['start'].year)*12 + (row['end'].month - row['start'].month)
                        return row

                    stmnt_df = stmnt_df.apply(lambda row: get_time_period_months(row), axis= 1)

                    # filtering out records that are not within the last history_years --IMPORTANT
                    stmnt_df = stmnt_df[stmnt_df['start'] > (datetime.now() - timedelta(days=366*history_years)).date()]

                    # filter out rows that are not quarterly/yearly
                    stmnt_df = stmnt_df[stmnt_df['time_period_months'].isin([2, 3, 4]) | stmnt_df['time_period_months'].isin([10, 11, 12])]
                    stmnt_df = stmnt_df[["start", "end", "fp", "time_period_months", "val", "filed", "orig_stmnt"]]   

                    #union with main df
                    stmnts_combined_df_op = pd.concat([stmnts_combined_df_op, stmnt_df], ignore_index=True)
            except KeyError as ke:
                # logger.error(f"KeyError: {ke}")
                continue
    return stmnts_combined_df_op

# @InvokeCorrespondingDeco.invoke_corresponding
def remove_inconsistencies(stmnts_combined_df: pd.DataFrame) -> pd.DataFrame:
    # Revenues can be net/total/segment --only getting the total ones
    stmnts_combined_df_cleansed = stmnts_combined_df.groupby(["start", "end", "fp", "filed", "time_period_months"]).agg({"val": "max"}).reset_index()

    # Filtering out ambigous cases and returning the df
    return stmnts_combined_df_cleansed[~stmnts_combined_df_cleansed['fp'].isnull() & ~((stmnts_combined_df_cleansed['time_period_months'] < 10) & (stmnts_combined_df_cleansed['fp'] == "FY")) & ~((stmnts_combined_df_cleansed['time_period_months'] > 4) & (stmnts_combined_df_cleansed['fp'].str.startswith("Q")))]

# @InvokeCorrespondingDeco.invoke_corresponding
def save_latest_q1_q3(stmnts_combined_df_cleansed: pd.DataFrame) -> pd.DataFrame:
    # save the latest q1-q3 for union later
    max_end_fy = (
        stmnts_combined_df_cleansed[stmnts_combined_df_cleansed['fp'] == "FY"]
        .max()["end"]
    )
    latest_q1_q3 = stmnts_combined_df_cleansed[stmnts_combined_df_cleansed['end'] > max_end_fy].copy()
    if not latest_q1_q3.empty:
        latest_q1_q3.loc[:, 'q1_q3_fy_rel'] = None  
        return latest_q1_q3
    cols = list(stmnts_combined_df_cleansed.columns) + ["q1_q3_fy_rel"]
    return pd.DataFrame(columns=cols)

# @InvokeCorrespondingDeco.invoke_corresponding
def assign_relation_between_q_and_fy(stmnts_combined_df_cleansed: pd.DataFrame) -> pd.DataFrame:
    fy_rows = stmnts_combined_df_cleansed[stmnts_combined_df_cleansed['fp'] == "FY"].rename(
        columns = {"start": "fy_start", "end": "fy_end"}
    )[['fy_start', 'fy_end']]

    stmnts_combined_df_with_rel = (
        pd.merge(stmnts_combined_df_cleansed, fy_rows, how="cross", suffixes=("", ""))
    )
    stmnts_combined_df_with_rel = stmnts_combined_df_with_rel[(stmnts_combined_df_with_rel['start'] >= stmnts_combined_df_with_rel['fy_start']) & (stmnts_combined_df_with_rel['end'] <= stmnts_combined_df_with_rel['fy_end'])]

    def add_relationship_col(row):
        row['q1_q3_fy_rel'] = str(row['fy_start']) + " " + str(row["fy_end"])
        return row
    
    stmnts_combined_df_with_rel = stmnts_combined_df_with_rel.apply(lambda row: add_relationship_col(row), axis= 1)

    return stmnts_combined_df_with_rel

# @InvokeCorrespondingDeco.invoke_corresponding
def get_q4_rows(stmnts_combined_df_with_rel: pd.DataFrame) -> pd.DataFrame:
    def q4_rows(grp):
        grp['q4_val'] = grp[(grp['fp'] == "FY")]['val'].values[0] - grp[~(grp['fp'] == "FY")]['val'].sum()
        grp['q4_start'] = grp[grp['fp'] == "Q3"]['end'] + timedelta(days=1)
        grp['q4_end'] = grp['fy_end']
        grp['q4_filed'] = grp[grp['fp'] == "FY"]['filed'].max()
        grp['q1_q3_fy_rel'] = str(grp['fy_start'].values[0]) + " " + str(grp["fy_end"].values[0])
        return grp

    q4_rows_df = stmnts_combined_df_with_rel.groupby('q1_q3_fy_rel', group_keys=False).apply(lambda grp: q4_rows(grp))
    q4_rows_df = q4_rows_df.dropna(subset= ['q4_start', 'q4_end', 'q4_filed'])
    q4_rows_df = q4_rows_df[['q4_start', 'q4_end', 'q4_filed', 'q4_val', 'q1_q3_fy_rel']].drop_duplicates().rename(columns = {'q4_start': 'start', 'q4_end': 'end', 'q4_filed': 'filed', 'q4_val': 'val'})
    q4_rows_df['fp'] = 'Q4'

    return q4_rows_df

# @InvokeCorrespondingDeco.invoke_corresponding
def get_fy_all_q_union_df(stmnts_combined_df_with_rel: pd.DataFrame, latest_q1_q3: pd.DataFrame, q4_rows_df: pd.DataFrame) -> pd.DataFrame:
    fy_all_q_union_df = pd.concat(
       [
        stmnts_combined_df_with_rel[["start", "end", "fp", "val", "filed", "q1_q3_fy_rel"]],
        q4_rows_df[["start", "end", "fp", "val", "filed", "q1_q3_fy_rel"]],
        latest_q1_q3[["start", "end", "fp", "val", "filed", "q1_q3_fy_rel"]]
       ]
    )
    return fy_all_q_union_df

# @InvokeCorrespondingDeco.invoke_corresponding   
def period_corrected(fy_all_q_union_df: pd.DataFrame) -> pd.DataFrame:
    def get_period_val(row):
        if row['q1_q3_fy_rel'] is None:
            if row['fp'].startswith("Q"):
                fy_rows_df = fy_all_q_union_df[fy_all_q_union_df['fp'] == 'FY'].copy()
                row['Period'] = row['fp'][::-1] + str(int(str(fy_rows_df['end'].max()).split("-")[0][-2:]) + 1)
        else:
            if row['fp'] == "FY":
                row['Period'] = row['fp'] + row['q1_q3_fy_rel'].split(" ")[1].split("-")[0][-2:]
            elif row['fp'].startswith("Q"):
                row['Period'] = row['fp'][::-1] + row['q1_q3_fy_rel'].split(" ")[1].split("-")[0][-2:]
        return row
    return fy_all_q_union_df.apply(lambda row: get_period_val(row), axis=1)

# @InvokeCorrespondingDeco.invoke_corresponding
def get_base_sheet_format(fy_all_q_union_df: pd.DataFrame, Ticker: str) -> pd.DataFrame:
    # base sheet format
    fy_all_q_union_df['Ticker'] = Ticker + " " + "US Equity"
    fy_all_q_union_df['Period_Startdate'] = fy_all_q_union_df['start'].apply(lambda d: f"{d.month}/{d.day}/{d.year}")
    fy_all_q_union_df['Period_Enddate'] = fy_all_q_union_df['end'].apply(lambda d: f"{d.month}/{d.day}/{d.year}")
    fy_all_q_union_df['Period_Reportdate'] = fy_all_q_union_df['filed'].apply(lambda d: f"{d.month}/{d.day}/{d.year}")
    fy_all_q_union_df["Value"] = fy_all_q_union_df['val'].apply(lambda x: "{:,}".format(x))
    fy_all_q_union_df["KPI"] = "rev_Topline"
    fy_all_q_union_df["_"] = ""
    fy_all_q_union_df["Segment_Identifier"] = ""
    fy_all_q_union_df["Revison_Date"] = ""
    fy_all_q_union_df["Original_Value"] = ""
    fy_all_q_union_df["Source"] = fy_all_q_union_df.apply(lambda row: "10-K " + row['Period'] if row['fp'] in ["FY", "Q4"] else "10-Q " + row['Period'], axis=1)
    
    fy_all_q_union_bsf_df = fy_all_q_union_df[[
            "Ticker",
            "Period",
            "Period_Startdate",
            "Period_Enddate",
            "Period_Reportdate",
            "_",
            "Segment_Identifier",
            "KPI",
            "Value",
            "Revison_Date",
            "Original_Value",
            "Source",
        ]]
    
    # ).filter((F.col("Period").rlike("Q|F")) & (F.col("val") > 0))
    return fy_all_q_union_bsf_df

# @InvokeCorrespondingDeco.invoke_corresponding
def get_not_in_base_sheet_rows(fy_all_q_union_df: pd.DataFrame, Ticker: str) -> pd.DataFrame:
    not_in_base_sheet_df = (
        F.broadcast(
            spark.createDataFrame(fy_all_q_union_df).withColumn(
                "period_sub_str", F.col("period").substr(0, 2)
            ).alias("a")
        )
        .join(
            table("bronze.internal_products.financials_cians_raw")
            .filter(F.col("ticker") == Ticker + " US Equity")
            .filter(F.col("kpi") == "rev_Topline")
            .filter(F.col("value").isNotNull())
            .select("period", "period_startdate", "period_enddate")
            .withColumn("period_sub_str", F.col("period").substr(0, 2))
            .alias("b"),
            on=(F.col("a.Period_Startdate") == F.col("b.period_startdate"))
            & (F.col("a.Period_Enddate") == F.col("b.period_enddate"))
            & (F.col("a.period_sub_str") == F.col("b.period_sub_str")),
            how="left_anti",
        )
        .drop("period_sub_str")
    )
    return not_in_base_sheet_df.toPandas()


In [0]:
cik_map_df = get_cik_lookup_df()

cik_map_df = cik_map_df[
    cik_map_df['ticker'].isin(get_input_tickers(tickers_str_with_region))
]
display(cik_map_df)

In [0]:
for Ticker in get_input_tickers(tickers_str_with_region= tickers_str_with_region):
    try:
        cik_map_df_filt = cik_map_df[cik_map_df['ticker'] == Ticker]['cik']
        cik = cik_map_df_filt.values[0]
    except Exception:
        logger.warning((f"{Ticker} not present in cik map. Check cmd 8 for more info"))
        rev_bif_not_found.append(Ticker + "-US")
        continue
    logger.info(f"Processing ticker {Ticker}, cik {cik}")
    processed_tickers.append(Ticker + "-US")
    comp_facts_dict = get_company_facts(cik= cik)
    stmnts_combined_df = create_stmnts_combined_df(comp_facts_dict= comp_facts_dict, history_years=history_years)
    #df must not be empty and should have atleast one FY row present otherwise assume financials not found for that particular ticker
    if  (not stmnts_combined_df.empty and bool(sum(stmnts_combined_df[stmnts_combined_df['fp'] == 'FY'].count().values))):
        stmnts_combined_df_cleansed = remove_inconsistencies(stmnts_combined_df)
        latest_q1_q3 = save_latest_q1_q3(stmnts_combined_df_cleansed= stmnts_combined_df_cleansed)
        stmnts_combined_df_with_rel = assign_relation_between_q_and_fy(
            stmnts_combined_df_cleansed= stmnts_combined_df_cleansed
        )
        q4_rows_df = get_q4_rows(stmnts_combined_df_with_rel= stmnts_combined_df_with_rel)
        fy_all_q_union_df = get_fy_all_q_union_df(
            stmnts_combined_df_with_rel= stmnts_combined_df_with_rel, latest_q1_q3= latest_q1_q3, q4_rows_df= q4_rows_df
        )
        fy_all_q_union_pc_df = period_corrected(fy_all_q_union_df= fy_all_q_union_df)
        fy_all_q_union_bsf_df = get_base_sheet_format(fy_all_q_union_pc_df, Ticker)
        if(include_historical_data):           
            final_result_df = pd.concat([final_result_df, fy_all_q_union_bsf_df])  
        else:
            not_in_base_sheet_df = get_not_in_base_sheet_rows(fy_all_q_union_bsf_df, Ticker)
            if not_in_base_sheet_df.empty:
                pass
            else:
                final_result_df = pd.concat([final_result_df, not_in_base_sheet_df])     
    else:
        logger.info(f"us-gaap financials not found for {Ticker}")
        rev_bif_not_found.append(Ticker + "-US")
        continue

In [0]:
final_result_df['Ticker_sort'] = final_result_df['Ticker'].str.lower()
final_result_df['Period_Enddate_sort'] = pd.to_datetime(final_result_df['Period_Enddate'])
final_result_df['Value_sort'] = pd.to_numeric(final_result_df['Value'].str.replace(',', ''))

if not final_result_df.empty:
    display(
        final_result_df.sort_values(
            by=['Ticker_sort', 'Period_Enddate_sort', 'Value_sort'],
            ascending=False
        ).drop(
            columns=['Ticker_sort', 'Period_Enddate_sort', 'Value_sort']
        )
    )
else:
    logger.info("No data in final_result_df")

In [0]:
if not final_result_df.empty:
    add_to_base_sheet = [
        str(ticker[0]).replace(" US Equity", "-US")
        for ticker in final_result_df[['Ticker']].drop_duplicates().values]
else:
    add_to_base_sheet = []

In [0]:
new_processed_tickers = [t.replace("-US", "") for t in set(processed_tickers)]
new_rev_bif_not_found = [t.replace("-US", "") for t in set(rev_bif_not_found)]

if set(get_input_tickers(tickers_str_with_region)) == set(new_processed_tickers):
    logger.info("No anomalies")
else:
    logger.warning(
        f"Anomalies found for {set(get_input_tickers(tickers_str_with_region)) - set(new_processed_tickers)}"
    )

In [0]:
print(("\n").join(set(processed_tickers) - set(add_to_base_sheet)))

In [0]:
dbutils.notebook.exit(
    f"\nFinancials for the following tickers were not found: {list(set(rev_bif_not_found))}\nFollowing tickers were processed: {processed_tickers}\nNew rows for these tickers can be added: {add_to_base_sheet}"
)

In [0]:
display(table('bronze.internal_products.financials_cians_raw').select('ticker').distinct())

In [0]:
table('bronze.internal_products.financials_cians_raw').columns

In [0]:
spark.sql(" select round(months_between('2024-10-01', '2024-12-31'),2) ").display()

In [0]:
# final_result_df.write.mode('overwrite').saveAsTable('silver_dev.dse_investigations.sec_gov_api_rev_bench_daily_workflow_tickers')

In [0]:
# display(table('silver_dev.dse_investigations.sec_gov_api_rev_bench_daily_workflow_tickers'))

In [0]:
print(table('bronze.internal_products.financials_cians_raw').columns)

In [0]:
from pandas import json_normalize
test_df = (spark.createDataFrame(json_normalize(comp_facts_dict['facts']['us-gaap']['RevenueFromContractWithCustomerIncludingAssessedTax']['units']))
        .withColumn('USD', F.explode(F.col('USD')))
        .select("USD.*")
        .withColumn('start', F.col('start').cast(DateType()))
        .withColumn('end', F.col('end').cast(DateType()))
        .withColumn("time_period_months", F.round(F.months_between(F.col('end'), F.col('start')), 2))
        .filter(" start > date_sub(current_date(), ) ")
        .filter(""" time_period_months between 2.7 and 4 or time_period_months between 11 and 13  """)
        # # .filter(F.col('time_period_months').between(11.50, 13))
        # .dropDuplicates(['start', 'end', 'val'])
        # .drop(*['description', 'label', 'accn', 'frame', 'fy'])
        # .orderBy(F.col('end').desc()))
)

display(test_df)

In [0]:
display(test_df.join(get_fiscal_year_quarters(test_df), ['start', 'end', 'time_period_months'], 'left'))

In [0]:
display(spark.createDataFrame(json_normalize(comp_facts_dict['facts']['us-gaap']['Revenues']['units'])).withColumn('USD', F.explode(F.col('USD'))).select("USD.*").withColumn('end', F.col('end').cast(DateType())).orderBy(F.col('end').desc()))

In [0]:
print([each for each in comp_facts_dict['facts']['us-gaap'].keys() if re.search("revenue", each.lower())])

In [0]:
display(comp_facts_dict['facts']['us-gaap'][key_to_use]['units']['USD'])

In [0]:
from pprint import pprint
pprint(comp_facts_dict['facts']['us-gaap'][key_to_use]['units']['USD'])